In [16]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

Завантанження та клінінг даних

In [45]:
# === pandas ===
start = time.time()
df = pd.read_csv("data.txt", sep=";", na_values="?", low_memory=False)
df.dropna(inplace=True)
end = time.time() - start
print(f"=== pandas === {end:.3f} с")

# === numpy ===

start = time.time()
types = [
    ("Date", "U10"),
    ("Time", "U8"),
    ("Global_active_power", "f8"),
    ("Global_reactive_power", "f8"),
    ("Voltage", "f8"),
    ("Global_intensity", "f8"),
    ("Sub_metering_1", "f8"),
    ("Sub_metering_2", "f8"),
    ("Sub_metering_3", "f8")
]

arr = np.genfromtxt(
    "data.txt", delimiter=";", dtype=types, names=True,
    missing_values="?", filling_values=np.nan, encoding="utf-8"
)

np_time = time.time() - start
print(f"=== numpy === {np_time:.3f} с")


=== pandas === 2.053 с


FileNotFoundError: [Errno 2] No such file or directory: 'AirQualityUCI.csv'

In [44]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [43]:
arr

array([('16/12/2006', '17:24:00', 4.216, 0.418, 234.84, 18.4, 0., 1., 17.),
       ('16/12/2006', '17:25:00', 5.36 , 0.436, 233.63, 23. , 0., 1., 16.),
       ('16/12/2006', '17:26:00', 5.374, 0.498, 233.29, 23. , 0., 2., 17.),
       ...,
       ('26/11/2010', '21:00:00', 0.938, 0.   , 239.82,  3.8, 0., 0.,  0.),
       ('26/11/2010', '21:01:00', 0.934, 0.   , 239.7 ,  3.8, 0., 0.,  0.),
       ('26/11/2010', '21:02:00', 0.932, 0.   , 239.55,  3.8, 0., 0.,  0.)],
      shape=(2075259,), dtype=[('Date', '<U10'), ('Time', '<U8'), ('Global_active_power', '<f8'), ('Global_reactive_power', '<f8'), ('Voltage', '<f8'), ('Global_intensity', '<f8'), ('Sub_metering_1', '<f8'), ('Sub_metering_2', '<f8'), ('Sub_metering_3', '<f8')])

Завдання №1. 
Записи, у яких Global_active_power > 5 кВт.

In [ ]:
# === pandas ===
start = time.time()
df_t1 = df[df["Global_active_power"] > 5]
end = time.time() - start
print(f"=== pandas === {end:.3f} с")


# === numpy ===
start = time.time()
arr_t1 = arr[arr["Global_active_power"] > 5]
end = time.time() - start
print(f"=== numpy === {end:.3f} с")

In [ ]:
df_t1.head()

In [ ]:
arr_t1

Завдання №2. Записи, де Voltage > 235 В.

In [ ]:
# === pandas ===
start = time.time()
df_t2 = df[df["Voltage"] > 235]
end = time.time() - start
print(f"=== pandas === {end:.3f} с")

# === numpy ===
start = time.time()
arr_t2 = arr[arr["Voltage"] > 235]
end = time.time() - start
print(f"=== numpy === {end:.3f} с")

In [ ]:
df_t2.head()

In [ ]:
arr_t2

Завдання №3. Записи, у яких сила струму лежить в межах 19-20 А, та, у яких пральна машина та холодильних
споживають більше, ніж бойлер та кондиціонер.

In [30]:
# === pandas ===
start = time.time()
df_t3 = df[(df["Global_intensity"] >= 19) & (df["Global_intensity"] <= 20)]
df_t3 = df_t3[df_t3["Sub_metering_2"] > df_t3["Sub_metering_3"]]
end = time.time() - start
print(f"=== pandas === {end:.3f} с")

# === numpy ===
start = time.time()
arr_t3 = (arr["Global_intensity"] >= 19) & (arr["Global_intensity"] <= 20)
arr_t32 = arr["Sub_metering_2"] > arr["Sub_metering_3"]
arr_t3 = arr[arr_t3 & arr_t32]
end = time.time() - start
print(f"=== numpy === {end:.3f} с")

=== pandas === 0.012 с
=== numpy === 0.069 с


In [ ]:
df_t3.head()

In [ ]:
arr_t3

Завдання №4. Середні величини всіх 3-х груп споживання електричної енергії з випадково вибраних 500 000 записів  



In [ ]:
# === pandas ===
start = time.time()
sample = df.sample(n=500_000, replace=False)
df_t4 = sample[["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]].mean()
end = time.time() - start
print(df_t4)
print(f"{end:.3f} с\n")

# === numpy ===
start = time.time()
random_indices = np.random.choice(len(arr), size=500_000, replace=False)
sample = arr[random_indices]
arr_t4 = {
    "Sub_metering_1": np.mean(sample["Sub_metering_1"]),
    "Sub_metering_2": np.mean(sample["Sub_metering_2"]),
    "Sub_metering_3": np.mean(sample["Sub_metering_3"])
}
end = time.time() - start
print(arr_t4)
print(f"{end:.3f} с")


=== pandas === Середні значення для 500000 випадкових записів:
Sub_metering_1    1.131242
Sub_metering_2    1.298606
Sub_metering_3    6.443620
dtype: float64
0.216 с

{'Sub_metering_1': np.float64(nan), 'Sub_metering_2': np.float64(nan), 'Sub_metering_3': np.float64(nan)}
0.181 с


Завдання №5. Записи, які:
- після 18-00 споживають більше 6 кВт за хвилину в середньому
- у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення
- обраний кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [39]:
# === pandas ===
start = time.time()

evening = df[df["Time"] >= pd.to_datetime("18:00:00").time()]
pivo = evening[evening["Global_active_power"] > 6]
combo = pivo[
    (pivo["Sub_metering_2"] > pivo["Sub_metering_1"]) &
    (pivo["Sub_metering_2"] > pivo["Sub_metering_3"])
]

mid = len(combo) // 2
half1 = combo.iloc[:mid]
half2 = combo.iloc[mid:]
df_3rd = half1.iloc[::3]
df_4rd = half2.iloc[::4]
df_t5 = pd.concat([df_3rd, df_4rd])
end = time.time() - start
print(f"=== pandas === {end:.3f} с")

# === numpy ===
start = time.time()

def hour(tstr):
    return int(tstr.split(":")[0])

hours = np.array([hour(t) for t in arr["Time"]])
evening = hours >= 18
pivo = arr["Global_active_power"] > 6
combo = evening & pivo

filt = arr[combo]
groupn = (
    (filt["Sub_metering_2"] > filt["Sub_metering_1"]) &
    (filt["Sub_metering_2"] > filt["Sub_metering_3"])
)
filtn = filt[groupn]

mid = len(filtn) // 2
np_3rd = filtn[:mid][::3]
np_4rd = filtn[mid:][::4]
arr_t5 = np.concatenate([np_3rd, np_4rd])

end = time.time() - start
print(f"=== numpy === {end:.3f} с")


=== pandas === 0.131 с
=== numpy === 1.428 с


### Результати експерименту

| №  | Опис завдання                                                                                                     | pandas Час (с) | numpy Час (с)  | Оцінка pandas | Оцінка numpy | Коментар                                                                 |
|----|-------------------------------------------------------------------------------------------------------------------|------------------|------------------|------------------|------------------|--------------------------------------------------------------------------|
| 1  | Завантаження та очищення даних                                                                                    | 2.122            | 11.836           | 5                | 2                | pandas значно швидше і простіше для очищення                           |
| 2  | Відбір Global_active_power > 5 кВт                                                                              | 0.009            | 0.026            | 5                | 4                | pandas зручніше, numpy трохи повільніше                              |
| 3  | Відбір Voltage > 235 В                                                                                          | 0.179            | 0.208            | 4                | 4                | обидві структури показали подібну ефективність                           |
| 4  | Струм 19–20 А, sub_metering_2 > sub_metering_3                                                                  | 0.013            | 0.056            | 5                | 3                | pandas простіше для комбінованих умов                                  |
| 5  | Середні значення 3-х sub_metering для 500 000 випадкових записів                                                | 0.268            | 0.214            | 4                | 5                | numpy швидший при числовій агрегації                                   |
| 6  | Фільтрація за часом, потужністю, приладами, з вибіркою кожного 3-го/4-го запису                                   | 6.442            | 1.482            | 2                | 5                | numpy значно швидший у складних умовах       |
